In [8]:
import pandas as pd
import os
from ast import literal_eval
from itertools import product
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/Superficie/2_ProteinOrtho/3_Filtrado')

In [9]:
orto_df = pd.read_csv('Mus_Dic_Orto.csv') 
orto_df.head()

,Unnamed: 0,Model_nodos,NCBI_Product_accesion,Astyanax_ortologos,ID
0,0,ABCA1,XP_006537619.1,"[('XP_022522959.2', 'ABCA1A'), ('XP_022522960....",1
1,1,ABCB11,NP_066302.2,"[('XP_022525386.2', 'ABCB11B')]",2
2,2,ABCB1A,NP_035206.2,"[('XP_049320497.1', 'ABCB4')]",3
3,3,ABCB1B,NP_035205.1,"[('XP_049320497.1', 'ABCB4')]",4
4,4,ABCB4,NP_032856.2,"[('XP_049320497.1', 'ABCB4')]",5


In [10]:
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/Superficie/1_Redes_modelo')
net_df= pd.read_csv ('trrust_rawdata.mouse.csv')

In [11]:
net_df.head()

,Source,Target,Interaction
0,AATF,BAK1,?
1,AATF,BAX,?
2,AATF,BBC3,?
3,AATF,CDKN1A,?
4,AATF,TPT1,+


In [12]:

# Convertir string de listas de tuplas a listas reales
orto_df["Astyanax_ortologos"] = orto_df["Astyanax_ortologos"].apply(literal_eval)

# Crear diccionario para acceso rápido a ortólogos
orto_dict = orto_df.set_index("Model_nodos")[["Astyanax_ortologos", "ID"]].to_dict(orient="index")

# Generar combinatorias
expanded_rows = []

for _, row in net_df.iterrows():
    source = row["Source"]
    target = row["Target"]
    interaction = row["Interaction"]
    
    if source in orto_dict and target in orto_dict:
        source_orthos = orto_dict[source]["Astyanax_ortologos"]
        target_orthos = orto_dict[target]["Astyanax_ortologos"]
        id_source = orto_dict[source]["ID"]
        id_target = orto_dict[target]["ID"]
        
        for s_ort, t_ort in product(source_orthos, target_orthos):
            expanded_rows.append({
                "Source_ortolog": s_ort[1],
                "Target_ortolog": t_ort[1],
                "Interaction": interaction,
                "ID_source": id_source,
                "ID_target": id_target,
                "Grupo_interaccion": f"{id_source}-{id_target}"
            })

# Crear dataframe final
final_df = pd.DataFrame(expanded_rows)


In [13]:
final_df.head()

,Source_ortolog,Target_ortolog,Interaction,ID_source,ID_target,Grupo_interaccion
0,AIRE,BMP2A,?,35,94,35-94
1,ANKRD1A,MYL2B,-,48,648,48-648
2,ANKRD1A,MYL2A,-,48,648,48-648
3,APBB1,LRP1AB,-,51,572,51-572
4,APBB1,LRP1AB,-,51,572,51-572


In [14]:
final_df.shape

(6110, 6)

In [15]:
final_df.to_csv('Mus_net_comb.csv')